In [1]:
from typing import List
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, StateGraph, MessagesState

# LLMモデルの設定
local_path = "gguf_models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf"
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path=local_path,
    callback_manager=callback_manager,
    verbose=False
)

In [2]:
class AgentState(MessagesState):
    agent_name: str
    other_agent_name: str

def create_agent_state(name: str, other_name: str) -> AgentState:
    return AgentState(messages=[], agent_name=name, other_agent_name=other_name)

In [3]:
def generate_response(state: AgentState):
    messages = state['messages']
    prompt = f"あなたは{state['agent_name']}です。{state['other_agent_name']}と会話をしています。以下の会話を続けてください：\n\n"
    for message in messages:
        if isinstance(message, HumanMessage):
            prompt += f"{state['other_agent_name']}: {message.content}\n"
        elif isinstance(message, AIMessage):
            prompt += f"{state['agent_name']}: {message.content}\n"
    
    response = llm.invoke(prompt)
    return {"messages": [AIMessage(content=response)]}

In [4]:
def create_conversation_graph(agent1_name: str, agent2_name: str):
    workflow = StateGraph(AgentState)
    
    workflow.add_node("agent1", lambda x: generate_response(x))
    workflow.add_node("agent2", lambda x: generate_response(x))
    
    workflow.set_entry_point("agent1")
    
    workflow.add_edge("agent1", "agent2")
    workflow.add_edge("agent2", "agent1")
    
    return workflow.compile()

# 会話グラフの作成
conversation = create_conversation_graph("Alice", "Bob")

In [5]:
def run_conversation(turns: int):
    alice_state = create_agent_state("Alice", "Bob")
    bob_state = create_agent_state("Bob", "Alice")
    
    alice_state['messages'].append(HumanMessage(content="こんにちは、Bob。今日はいい天気ですね。"))
    
    for _ in range(turns):
        alice_result = conversation.invoke(alice_state)
        alice_state['messages'].extend(alice_result['messages'])
        bob_state['messages'].extend(alice_result['messages'])
        
        bob_result = conversation.invoke(bob_state)
        bob_state['messages'].extend(bob_result['messages'])
        alice_state['messages'].extend(bob_result['messages'])
        
        print(f"Alice: {alice_result['messages'][-1].content}")
        print(f"Bob: {bob_result['messages'][-1].content}")
        print()

# 会話の実行
run_conversation(5)  # 5ターンの会話を実行

Alice: こんにちは、Bob。本当に良い天気です。花見も良さそうですね。

Bob: 花見は来週の予定です。でも、良い天気なので、急遽ピクニックを計画しました。
Alice: ピクニックは楽しいですね！私はお弁当を作って持って行きます。ボブさんは何を持って行きますか？

Bob: それは良く考えています。実は、近所に新しく出来たパン屋さんがあるのですが、そこで美味しそうなバゲットを買って持って行こうと思っています。

Alice: それがとても素敵ですね！私はお弁当とお茶を持って行きます。ピクニックは計画する段階から楽しいですよね！

Bob: はい、本当にその通りです。ピクニックの計画をしている時から、ワクワクして楽しみです。実際にピクニックに行っても、自然の中で美味しい食事や会話を楽しむことができます。とても有意義な時間を過ごすことができ、また次回のピクニックも計画Posted on 2021年9月21日 by Alice
This entry was posted in Diary and tagged Bob, Alice, ピクニック. Bookmark the permalink.
← BobとAliceの会話：花見 → BobとAliceの会話：新しいパン屋さん →

Leave a reply Cancel reply
You are commenting using your WordPress.com account. ( Log Out / Change)

Notify me of follow-up comments by email.

Notify me of new posts by email.

Your email address will not be published. Required fields are marked *.Comment *
Name *
Email * (optional)
Website *

More information:

* You can use these HTML tags and attributes: <a href="" title=""> <abbr title=""> <acronym title=""> <b> <blockquote cite=""> <cite>

ValueError: Requested tokens (514) exceed context window of 512